# Nepali GPT Experiment

First line of devkota's Bhikhari poem:

 हेर भिखारी अडि अडि आयो

In [11]:
text = "हेर भिखारी अडि अडि आयो"
print(text)
ltext = list(text)
len(text)
print(ltext)

हेर भिखारी अडि अडि आयो
['ह', 'े', 'र', ' ', 'भ', 'ि', 'ख', 'ा', 'र', 'ी', ' ', 'अ', 'ड', 'ि', ' ', 'अ', 'ड', 'ि', ' ', 'आ', 'य', 'ो']


If you count the number of characters in the above line of text, there are actually `15` user-perceived characters. But when we use the `len` method on that text, it returns `22`.

Non-english texts especially the ones that may make use of multiple Unicode codepoint to represent a single user-perceived character cannot be split using Python's built-in text splitting routines. They need special functions that are aware of graphemes.

Look at: https://github.com/alvinlindstam/grapheme/tree/master

Following has some good discussion on the grapheme cluster algorithm: https://bugs.python.org/issue30717

In [8]:
import grapheme

In [13]:
grapheme.length(text)
gs = grapheme.graphemes(text)
for g in gs:
    print(g)

हे
र
 
भि
खा
री
 
अ
डि
 
अ
डि
 
आ
यो


If you notice, `grapheme` module above correctly returns `15`.

Devanagari Unicode: https://unicode.org/charts/PDF/U0900.pdf

In [18]:
new_text = "ज्ञानीत्रक्ष"
new_list = list(new_text)
print(new_list)

['ज', '्', 'ञ', 'ा', 'न', 'ी', 'त', '्', 'र', 'क', '्', 'ष']


In [19]:
for gr in grapheme.graphemes(new_text):
    print(gr)

ज्
ञा
नी
त्
र
क्
ष


Graphemes works better than just using single Unicode codepoint. Even then, since some characters that are normally treated as a single unit are not defined in the standard, it can still be confusing. For now, we will work with whatever we have.

We will using grapheme as the single unit to construct the language model.

In [39]:
import os
f = open('../devkota/bhikari.txt', 'r')
txt = f.read()

In [24]:
print(txt)

(१)
हेर भिखारी अडि अडि आयो
करुण दृष्टिले नजर उठायो।
गाढा दुखको मौन प्रकाश।
झिना आशा-तार बजायो
घाम उज्यालो आँगन पास।
एक बिन्दुमा गोल खसायो
जीवनको इतिहास।
(२)
हेर, हेर ती झुत्रा चिथरा
हाय! हे समय निष्ठुर।
जीवनप थमा बिचार पथरा!
काँपिरहेको थुरथुर।
 झल्लर झोली बढाउँछ, बबुरा।
करले अस्थिर, कातर।
(३)
बर्ष-बर्षका हेर तुषारा 
शिरको उपर गिरेको।
 हेर आँसुका खहरेहरूका
 मुखमा खोंच परेको,
 दिन-दिनको त्यो छातीमाथी 
चिरा चारक्क चिरेको।
(४)
अडि अडि धरमर स्वाँ स्वाँ गर्दै 
मौन विलौना वरिपरी भर्दै 
आर्तनादका हृदय फुटी,
 जड लट्ठीको भरमा पर्दै
 भन्दछ स्वरले छाती चिर्दै 
चामल एक मुठी।
 जीवनभरको एक पुकारा 
चामल एक मुठी।
(५)
 मानिससँगमा मानिसको यो 
अन्तर्दिलको रोदन। 
भाईहरुसँग भिक्षाको यो 
मुठी द याको याचना। 
घाम उज्यालो आँगनमा यो 
एक आँध्यारो अवलोकन।
 गुलबहरूको हाँसोबिच यो 
एक उन्यौँको रोदन।
(६)
 को होला यो, कसको छोरा?
कसको बाबु गरीब? 
कुन आमाले काख लिँदामा 
बल्थे दृगका दुइ दीप?
 कुन आशाले नजर खुलायो 
सूर्य चाद्रको नजरसमिप?
 किन मुर्झायो? किन वौलायो?
 किन मधुरो यो जीवन्- व्दीप? 
(७)
बुद्ध देवको नजर अगाडि
 यही

In [25]:
lines = txt.splitlines()

In [27]:
lines[0]

'(१)'

In [28]:
for line in lines:
    print(line)

(१)
हेर भिखारी अडि अडि आयो
करुण दृष्टिले नजर उठायो।
गाढा दुखको मौन प्रकाश।
झिना आशा-तार बजायो
घाम उज्यालो आँगन पास।
एक बिन्दुमा गोल खसायो
जीवनको इतिहास।
(२)
हेर, हेर ती झुत्रा चिथरा
हाय! हे समय निष्ठुर।
जीवनप थमा बिचार पथरा!
काँपिरहेको थुरथुर।
 झल्लर झोली बढाउँछ, बबुरा।
करले अस्थिर, कातर।
(३)
बर्ष-बर्षका हेर तुषारा 
शिरको उपर गिरेको।
 हेर आँसुका खहरेहरूका
 मुखमा खोंच परेको,
 दिन-दिनको त्यो छातीमाथी 
चिरा चारक्क चिरेको।
(४)
अडि अडि धरमर स्वाँ स्वाँ गर्दै 
मौन विलौना वरिपरी भर्दै 
आर्तनादका हृदय फुटी,
 जड लट्ठीको भरमा पर्दै
 भन्दछ स्वरले छाती चिर्दै 
चामल एक मुठी।
 जीवनभरको एक पुकारा 
चामल एक मुठी।
(५)
 मानिससँगमा मानिसको यो 
अन्तर्दिलको रोदन। 
भाईहरुसँग भिक्षाको यो 
मुठी द याको याचना। 
घाम उज्यालो आँगनमा यो 
एक आँध्यारो अवलोकन।
 गुलबहरूको हाँसोबिच यो 
एक उन्यौँको रोदन।
(६)
 को होला यो, कसको छोरा?
कसको बाबु गरीब? 
कुन आमाले काख लिँदामा 
बल्थे दृगका दुइ दीप?
 कुन आशाले नजर खुलायो 
सूर्य चाद्रको नजरसमिप?
 किन मुर्झायो? किन वौलायो?
 किन मधुरो यो जीवन्- व्दीप? 
(७)
बुद्ध देवको नजर अगाडि
 यही

In [29]:
lines[1].split()

['हेर', 'भिखारी', 'अडि', 'अडि', 'आयो']

In [35]:
for word in lines[3].split():
    for g in grapheme.graphemes(word):
        print(g)

गा
ढा
दु
ख
को
मौ
न
प्
र
का
श
।


In Nepali, there are twelve vowels and 33 consonant. Each 33 consonant can then have 12 different variation. That means there are in total (12 + 33 * 12) consonant variation. Counting all the vowels, there are (12 * 33) user perceived characters.

In [50]:
num_vowels = 12
num_consonants = 33
num_variations = 12
num_digits=10
total_user_perceived_chars = num_vowels + num_digits + num_consonants + num_consonants*num_variations
print(total_user_perceived_chars)

451


In [40]:
import torch

In [41]:
t = torch.randn(5)

In [42]:
t

tensor([-0.2630, -3.0752,  0.2395, -0.2381, -1.2377])

In [89]:
nep_consonants = "क ख ग घ ङ च छ ज झ ञ ट ठ ड ढ ण त थ द ध न प फ ब भ म य र ल व श ष स ह क्ष त्र ज्ञ"
nep_variation_chars = "् ा ि ी ु ू े ै ो ौ ं ः ँ ृ"
len(nep_consonants.split())
len(nep_variation_chars.split())

nep_graphemes = nep_consonants.split()
for nc in nep_consonants.split():
    for nvc in nep_variation_chars.split():
        composite_ch = nc+nvc
        nep_graphemes.append(composite_ch)

nep_vowels_graphemes = "अ आ इ ई उ ऊ ए ऐ ओ औ अं अः"
# 'extend' appends individual elements from one list to the other
nep_graphemes.extend(nep_vowels_graphemes.split())

nep_digits = "० १ २ ३ ४ ५ ६ ७ ८ ९"
nep_graphemes.extend(nep_digits.split())

nep_special_graphemes = "ऋ । ( ) ? - ! :"
nep_graphemes.extend(nep_special_graphemes.split())
print(nep_graphemes)

['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'क्ष', 'त्र', 'ज्ञ', 'क्', 'का', 'कि', 'की', 'कु', 'कू', 'के', 'कै', 'को', 'कौ', 'कं', 'कः', 'कँ', 'कृ', 'ख्', 'खा', 'खि', 'खी', 'खु', 'खू', 'खे', 'खै', 'खो', 'खौ', 'खं', 'खः', 'खँ', 'खृ', 'ग्', 'गा', 'गि', 'गी', 'गु', 'गू', 'गे', 'गै', 'गो', 'गौ', 'गं', 'गः', 'गँ', 'गृ', 'घ्', 'घा', 'घि', 'घी', 'घु', 'घू', 'घे', 'घै', 'घो', 'घौ', 'घं', 'घः', 'घँ', 'घृ', 'ङ्', 'ङा', 'ङि', 'ङी', 'ङु', 'ङू', 'ङे', 'ङै', 'ङो', 'ङौ', 'ङं', 'ङः', 'ङँ', 'ङृ', 'च्', 'चा', 'चि', 'ची', 'चु', 'चू', 'चे', 'चै', 'चो', 'चौ', 'चं', 'चः', 'चँ', 'चृ', 'छ्', 'छा', 'छि', 'छी', 'छु', 'छू', 'छे', 'छै', 'छो', 'छौ', 'छं', 'छः', 'छँ', 'छृ', 'ज्', 'जा', 'जि', 'जी', 'जु', 'जू', 'जे', 'जै', 'जो', 'जौ', 'जं', 'जः', 'जँ', 'जृ', 'झ्', 'झा', 'झि', 'झी', 'झु', 'झू', 'झे', 'झै', 'झो', 'झौ', 'झं', 'झः', 'झँ', 'झृ', 'ञ्', 'ञा', 'ञि', 'ञी', 'ञु', 'ञू', 'ञे', 'ञै', 'ञो', 'ञौ

In [93]:
sent = "दु:खको दिलामा ईश्वर बोल्दछ"

for g in grapheme.graphemes(sent):
    print(g)

दु
:
ख
को
 
दि
ला
मा
 
ई
श्
व
र
 
बो
ल्
द
छ


In [94]:
print(':' == ':')

True
